# BackEnd integration test : titanic

##### In this notebook we will test backend fonction on titanic data

In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import shap
import pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

First : Import titanic data

In [5]:
titanic = pd.read_csv('../data/titanic.csv', index_col='PassengerId')

In [4]:
#titanic = pd.read_pickle('../data/clean_titanic.pkl')

Training a OneHot encoding on categorical data

In [6]:
feature_num = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
feature_cat = ['Sex', 'Embarked']

In [7]:
#preprocessing = ce.OrdinalEncoder(cols=feature_cat)
preprocessing = ce.OneHotEncoder(cols=feature_cat)

In [8]:
X = titanic[feature_num+feature_cat]
y = titanic['Pclass']

In [9]:
X_encode = preprocessing.fit_transform(X)

## Séparation entraînement/test
Ici, on garde de côté un jeu de test à part dès le début pour évaluer les performances finales. L'argument `stratify` s'assure que le nombre de survivants est bien proportionnel au nombre de voyageurs dans chaque jeu de données. 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encode, 
    y, 
    stratify=y, 
    test_size=0.5,
    shuffle=True
)

In [11]:
model = LGBMClassifier()

In [12]:
model.fit(X_train, y_train);

In [13]:
y_pred = model.predict(X_test)

In [14]:
(y_test == y_pred).mean()

1.0

In [15]:
explainer = shap.TreeExplainer(model)

/home/74242d-fed/.conda/envs/Diaphane/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/74242d-fed/.conda/envs/Diaphane/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.iforest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/74242d-fed/.conda/envs/Diaphane/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.t

# We got encoded data, we got the encoding, we got model, let's start testing function.

Calcul contribution

In [16]:
from xplainable.decomposition.contributions import compute_contributions

In [17]:
contributions , bias = compute_contributions(x=X_test,
                      explainer=explainer)

In [18]:
contributions[0].head(3)

,Pclass,Age,SibSp,Parch,Fare,Sex_1,Sex_2,Embarked_1,Embarked_2,Embarked_3,Embarked_4
PassengerId,,,,,,,,,,,
340,8.361147,3.417535e-14,-3.361027e-17,-2.385245e-18,1.480998e-14,0.0,0.0,1.734723e-18,-1.209753e-15,0.0,0.0
268,-2.671565,-2.868105e-14,-1.856154e-16,-8.673617e-18,-1.314920e-14,0.0,0.0,-4.336809e-19,-3.406130e-15,0.0,0.0
374,8.361147,-6.505213e-17,-5.225854e-17,-2.385245e-18,1.930964e-14,0.0,0.0,-3.469447e-18,9.013623e-15,0.0,0.0


Inverse transform

In [19]:
from xplainable.utils.transform import inverse_transform

In [20]:
X_train_inverse = inverse_transform(x_init=X_test,
                                    preprocessing=preprocessing)

In [21]:
X_train_inverse.head(3)

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
PassengerId,,,,,,,
340,1,45,0,0,35.5000,male,S
268,3,25,1,0,7.7750,male,S
374,1,22,0,0,135.6333,male,C


Inverse transform contribution

In [22]:
from xplainable.decomposition.contributions import inverse_transform_contributions

In [23]:
inv_transf_contrib = inverse_transform_contributions(contributions=contributions[0],
                                preprocessing=preprocessing)

In [24]:
inv_transf_contrib.head(3)

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
PassengerId,,,,,,,
340,8.361147,3.417535e-14,-3.361027e-17,-2.385245e-18,1.480998e-14,0.0,-1.208018e-15
268,-2.671565,-2.868105e-14,-1.856154e-16,-8.673617e-18,-1.314920e-14,0.0,-3.406563e-15
374,8.361147,-6.505213e-17,-5.225854e-17,-2.385245e-18,1.930964e-14,0.0,9.010154e-15


In [25]:
X_train_inverse.shape

(446, 7)

In [26]:
contributions[0].shape

(446, 11)

Sorted Contribution and feature

In [27]:
from xplainable.decomposition.contributions import rank_contributions

In [28]:
contrib_ord, modal_ord, feature_ord = rank_contributions(s=inv_transf_contrib,
                   x=X_train_inverse)

In [29]:
contrib_ord.head(3)

,contribution_0,contribution_1,contribution_2,contribution_3,contribution_4,contribution_5,contribution_6
PassengerId,,,,,,,
340,8.361147,3.417535e-14,1.480998e-14,-1.208018e-15,-3.361027e-17,-2.385245e-18,0.0
268,-2.671565,-2.868105e-14,-1.314920e-14,-3.406563e-15,-1.856154e-16,-8.673617e-18,0.0
374,8.361147,1.930964e-14,9.010154e-15,-6.505213e-17,-5.225854e-17,-2.385245e-18,0.0


In [30]:
feature_ord.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
340,0,1,4,6,2,3,5
268,0,1,4,6,2,3,5
374,0,4,6,1,2,3,5


In [31]:
modal_ord.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
340,1,45,35.5,S,0,0,male
268,3,25,7.775,S,1,0,male
374,1,135.633,C,22,0,0,male


## Filter

hide sign

In [32]:
from xplainable.manipulation.filters import sign_contributions

In [33]:
mask_sign = sign_contributions(df=contrib_ord)

In [34]:
mask_sign.head(3)

,contribution_0,contribution_1,contribution_2,contribution_3,contribution_4,contribution_5,contribution_6
PassengerId,,,,,,,
340,True,True,True,False,False,False,True
268,False,False,False,False,False,False,True
374,True,True,True,False,False,False,True


Hide feature

In [35]:
from xplainable.manipulation.filters import hide_contributions

In [36]:
mask_feat = hide_contributions(var_dict=feature_ord,
                               features_list=[1,6])

In [37]:
mask_feat.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
340,True,False,True,False,True,True,True
268,True,False,True,False,True,True,True
374,True,True,False,False,True,True,True


Value of the contribution

In [38]:
from xplainable.manipulation.filters import cap_contributions

In [39]:
mask_cap = cap_contributions(s=contrib_ord, 
                             threshold=0.1)

In [40]:
mask_cap.head(3)

,contribution_0,contribution_1,contribution_2,contribution_3,contribution_4,contribution_5,contribution_6
PassengerId,,,,,,,
340,True,False,False,False,False,False,False
268,True,False,False,False,False,False,False
374,True,False,False,False,False,False,False


Combinaison des masques

In [41]:
from xplainable.manipulation.filters import combine_masks

In [42]:
comb_mask = combine_masks(masks_list=[mask_sign,mask_feat,mask_cap])

In [43]:
comb_mask.head(3)

,contrib_1,contrib_2,contrib_3,contrib_4,contrib_5,contrib_6,contrib_7
PassengerId,,,,,,,
340,True,False,False,False,False,False,False
268,False,False,False,False,False,False,False
374,True,False,False,False,False,False,False


Keep only the best of the best we want to keep

In [44]:
from xplainable.manipulation.filters import cutoff_contributions

In [45]:
comb_top_mask = cutoff_contributions(mask=comb_mask,
                     k=2)

In [46]:
comb_top_mask.head(4)

,contrib_1,contrib_2,contrib_3,contrib_4,contrib_5,contrib_6,contrib_7
PassengerId,,,,,,,
340,True,False,False,False,False,False,False
268,False,False,False,False,False,False,False
374,True,False,False,False,False,False,False
544,False,False,False,False,False,False,False


Contribution of hidden

In [47]:
from xplainable.manipulation.mask import compute_masked_contributions

In [48]:
sum_masked = compute_masked_contributions(s_contrib=contrib_ord,
                                          name='contrib_masked',
                                          mask=comb_top_mask)

In [49]:
sum_masked.head(3)

,contrib_masked
PassengerId,
340,4.774132e-14
268,-2.671565e+00
374,2.820010e-14


Summarize

In [50]:
from xplainable.manipulation.summarize import summarize_el

In [51]:
summarize = summarize_el(df=contrib_ord,
                         mask=comb_top_mask,
                         prefix='temp')

In [52]:
summarize.head(5)

,temp1
340,8.361147
268,NaN
374,8.361147
544,NaN
818,NaN


Select some lines

In [53]:
from xplainable.manipulation.select_lines import select_lines

In [54]:
ind = select_lines(df=X_test,condition='Age > 50')

In [55]:
ind

[592,
 153,
 685,
 626,
 583,
 367,
 450,
 196,
 660,
 858,
 233,
 117,
 493,
 55,
 175,
 327,
 151,
 125,
 269,
 484,
 852,
 696,
 156,
 571,
 253,
 16,
 715,
 627,
 407,
 695,
 497,
 34,
 494,
 171,
 318]

Load and save with Pickle

In [56]:
from xplainable.utils.io import save_pickle
from xplainable.utils.io import load_pickle

In [57]:
save_pickle(preprocessing,'../data/encoder.pkl',protocol=pickle.HIGHEST_PROTOCOL)
save_pickle(contrib_ord,'../data/contrib.pkl',protocol=pickle.HIGHEST_PROTOCOL)

In [58]:
obj1 = load_pickle('../data/encoder.pkl')
obj2 = load_pickle('../data/contrib.pkl')

In [59]:
obj1

OneHotEncoder(cols=['Sex', 'Embarked'], drop_invariant=False,
              handle_missing='value', handle_unknown='value', return_df=True,
              use_cat_names=False, verbose=0)

In [60]:
obj2.head(3)

,contribution_0,contribution_1,contribution_2,contribution_3,contribution_4,contribution_5,contribution_6
PassengerId,,,,,,,
340,8.361147,3.417535e-14,1.480998e-14,-1.208018e-15,-3.361027e-17,-2.385245e-18,0.0
268,-2.671565,-2.868105e-14,-1.314920e-14,-3.406563e-15,-1.856154e-16,-8.673617e-18,0.0
374,8.361147,1.930964e-14,9.010154e-15,-6.505213e-17,-5.225854e-17,-2.385245e-18,0.0


## Smart Explainer

Init Smart Explainer

In [92]:
import sys
sys.path.append('../..')
from xplainable.explainer.smart_explainer import SmartExplainer

In [93]:
xpl = SmartExplainer()

Let calcul contribution

In [94]:
contributions , bias = compute_contributions(x=X_test,
                      explainer=explainer)

Compile and Filter for almost everything

First, Compile.

In [95]:
target = pd.DataFrame(y_pred,columns=['target'],index=X_test.index)

In [96]:
xpl.compile(contributions=contributions[0],
            preprocessing=preprocessing, 
            x_init=X_test,
            y_pred=target)

In [97]:
xpl.data['contrib_sorted'].head(2)

,contribution_0,contribution_1,contribution_2,contribution_3,contribution_4,contribution_5,contribution_6
PassengerId,,,,,,,
340,8.361147,3.417535e-14,1.480998e-14,-1.208018e-15,-3.361027e-17,-2.385245e-18,0.0
268,-2.671565,-2.868105e-14,-1.314920e-14,-3.406563e-15,-1.856154e-16,-8.673617e-18,0.0


In [98]:
xpl.data['var_dict'].head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
340,0,1,4,6,2,3,5
268,0,1,4,6,2,3,5


In [99]:
xpl.data['x_sorted'].head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
340,1,45,35.5,S,0,0,male
268,3,25,7.775,S,1,0,male


In [100]:
xpl.columns_dict

{0: 'Pclass',
 1: 'Age',
 2: 'SibSp',
 3: 'Parch',
 4: 'Fare',
 5: 'Sex_1',
 6: 'Sex_2',
 7: 'Embarked_1',
 8: 'Embarked_2',
 9: 'Embarked_3',
 10: 'Embarked_4'}

Second, Filter

In [101]:
xpl.filter(features_to_hide=[2,3],
           threshold=0.1,    
           positive=True, 
           max_contrib=2
            )

In [102]:
xpl.mask.head(2)

,contrib_1,contrib_2,contrib_3,contrib_4,contrib_5,contrib_6,contrib_7
PassengerId,,,,,,,
340,True,False,False,False,False,False,False
268,False,False,False,False,False,False,False


Masked Contribution

In [103]:
xpl.masked_contributions.head(2)

,masked
PassengerId,
340,4.774132e-14
268,-2.671565e+00


To pandas

In [104]:
xpl.to_pandas().head(2)

,target,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3,feature_4,value_4,contribution_4,feature_5,value_5,contribution_5
340,1,Pclass,1,8.36115,Age,45,3.41754e-14,Fare,35.5,1.481e-14,Sex_2,S,-1.20802e-15,SibSp,0,-3.36103e-17
268,3,Pclass,3,-2.67156,Age,25,-2.86811e-14,Fare,7.775,-1.31492e-14,Sex_2,S,-3.40656e-15,SibSp,1,-1.85615e-16


Summarize

In [105]:
xpl.data['summary'].head(2)

,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3,feature_4,value_4,contribution_4,feature_5,value_5,contribution_5
340,Pclass,1,8.36115,Age,45,3.41754e-14,Fare,35.5,1.481e-14,Sex_2,S,-1.20802e-15,SibSp,0,-3.36103e-17
268,Pclass,3,-2.67156,Age,25,-2.86811e-14,Fare,7.775,-1.31492e-14,Sex_2,S,-3.40656e-15,SibSp,1,-1.85615e-16


Export de l'objet en pickle

In [106]:
xpl.to_pickle('../data/params.pkl')

In [107]:
xpl_load = load_pickle('../data/params.pkl')

In [108]:
type(xpl_load)

dict

## Multi Decorator

In [109]:
from xplainable.explainer.smart_explainer import SmartExplainer

In [590]:
xpl2 = SmartExplainer()

start with contributions

In [591]:
contributions , bias = compute_contributions(x=X_test,
                      explainer=explainer)

In [592]:
X_encode = preprocessing.fit_transform(X)

In [593]:
xpl2.compile(contributions=contributions,
            preprocessing=preprocessing, 
            x_init=X_test,
            model=model,
            y_pred=pd.DataFrame(y_pred,columns=['target'],index=X_test.index))

In [594]:
xpl2.data['contrib_sorted'][0].head(2)

,contribution_0,contribution_1,contribution_2,contribution_3,contribution_4,contribution_5,contribution_6
PassengerId,,,,,,,
301,-2.671565,-6.672484e-14,-5.181619e-15,-2.400424e-16,0.0,0.0,0.0
656,-2.700766,-2.176427e-14,3.774758e-15,-1.399054e-15,0.0,0.0,0.0


In [595]:
xpl2.data['contrib_sorted'][1].head(2)

,contribution_0,contribution_1,contribution_2,contribution_3,contribution_4,contribution_5,contribution_6
PassengerId,,,,,,,
301,-2.297077,-5.536240e-14,-1.813393e-14,-1.226449e-15,-5.204170e-18,4.770490e-18,0.0
656,8.840122,2.249221e-14,2.814589e-16,1.569925e-16,-3.469447e-18,-1.951564e-18,0.0


In [596]:
xpl2.data['var_dict'][0].head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
301,0,1,4,3,2,5,6
656,0,1,4,3,2,5,6


In [597]:
xpl2.data['var_dict'][1].head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
301,0,1,4,3,6,5,2
656,0,4,3,1,5,6,2


In [598]:
xpl2.data['x_sorted'][0].head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
301,3,unknow,7.75,0,0,female,Q
656,2,24,73.5,0,2,male,S


In [599]:
xpl2.data['x_sorted'][1].head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
PassengerId,,,,,,,
301,3,unknow,7.75,0,Q,female,0
656,2,73.5,0,24,male,S,2


In [600]:
xpl2.columns_dict

{0: 'Pclass',
 1: 'Age',
 2: 'SibSp',
 3: 'Parch',
 4: 'Fare',
 5: 'Sex_1',
 6: 'Sex_2',
 7: 'Embarked_1',
 8: 'Embarked_2',
 9: 'Embarked_3',
 10: 'Embarked_4'}

Second, Filter

In [601]:
xpl2.filter(features_to_hide=[2,3],
           threshold=0.1,    
           positive=True, 
           max_contrib=2
            )

In [602]:
xpl2.mask[0].head(2)

,contrib_1,contrib_2,contrib_3,contrib_4,contrib_5,contrib_6,contrib_7
PassengerId,,,,,,,
301,False,False,False,False,False,False,False
656,False,False,False,False,False,False,False


In [603]:
xpl2.mask[1].head(2)

,contrib_1,contrib_2,contrib_3,contrib_4,contrib_5,contrib_6,contrib_7
PassengerId,,,,,,,
301,False,False,False,False,False,False,False
656,True,False,False,False,False,False,False


Masked Contribution

In [604]:
xpl2.masked_contributions[0].head(2)

,masked
PassengerId,
301,-2.671565
656,-2.700766


In [605]:
xpl2.masked_contributions[1].head(2)

,masked
PassengerId,
301,-2.297077e+00
656,2.292524e-14


Export result in dataframe

In [606]:
xpl2.to_pandas(features_to_hide=['Age','Pclass','Fare']).head(2)

,target,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3,feature_4,value_4,contribution_4
301,3,Sex_2,Q,2.894386e-15,Parch,0,6.505213e-19,SibSp,0,0.000000e+00,Sex_1,female,0.0
656,2,Parch,0,2.814589e-16,Sex_1,male,-3.469447e-18,Sex_2,S,-1.951564e-18,SibSp,2,0.0


Export de l'objet en pickle

In [607]:
xpl2.to_pickle('../data/params.pkl')

In [608]:
xpl_load = load_pickle('../data/params.pkl')

In [609]:
type(xpl_load)

dict

## Smart APP

In [616]:
import category_encoders as ce

In [617]:
import category_encoders as ce